# Capstone Clustering Week 3 Part 1

In [1]:
import numpy as np

In [2]:
import pandas as pd
import json

In [3]:
from geopy.geocoders import Nominatim

In [4]:
import requests

from bs4 import BeautifulSoup

In [5]:
from pandas.io.json import json_normalize

In [6]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [7]:
from sklearn.cluster import KMeans

In [9]:
!pip install folium
import folium

     |████████████████████████████████| 92kB 8.0MB/s eta 0:00:011


In [10]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [11]:
BS = BeautifulSoup(data, 'html.parser')

In [12]:
postalCode = []
borough = []
neighborhood = []

In [15]:
BS.find('table').find_all('tr')
BS.find('table').find_all('tr')
for row in BS.find('table').find_all('tr'):
    cells = row.find_all('td')
    
    if(len(cells) > 0):
        postalCode.append(cells[0].text)
        borough.append(cells[1].text)
        neighborhood.append(cells[2].text.rstrip('\n'))

In [16]:
neighbourhood_df = pd.DataFrame({"PostalCode": postalCode,
                           "Borough": borough,
                           "Neighborhood": neighborhood})

neighbourhood_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop cells with a borough that is "Not assigned"

In [17]:
neighbourhood_df_dropna = neighbourhood_df[neighbourhood_df.Borough != "Not assigned"].reset_index(drop=True)
neighbourhood_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


Group Neighbourhood

In [18]:
neighbourhood_df_group = neighbourhood_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
neighbourhood_df_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
neighbourhood_df_group.shape

(103, 3)

# Get Coordinates from Coursera

In [21]:
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
NB_df_new = neighbourhood_df_group.merge(coordinates, on="PostalCode", how="left")
NB_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
